# Analisis DataSet Properati segunda etapa

## Modelando con regresion lineal

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, LabelEncoder
from sklearn.metrics import r2_score
import statsmodels.api as sm

   De la etapa anterior de estudio obtenemos un df depurado y enriquecido al cual lo hemos guardado con el nombre de  **df_Properati_primer_limpieza.csv**

In [ ]:
df_properati2= pd.read_csv("df_Properati_primer_limpieza.csv")

### Objetivo de esta etapa:
Construir un modelo de regresion lineal capaz de predecir el precio por m2 de una propiedad en funcion de una serie de Features en el ambito de la Ciudad Autonoma de Buenos Aires (CABA). Para ello partimos de un DataFrame previamente estudiado y mejorado. En base a dicho analisis descartaremos informacion del mismo que consideramos irrelevante para esta etapa.

In [ ]:
#Recordamos las columnas
df_properati2.columns

Las columnas que eliminaremos en primer lugar son las siguientes                     

`'ID','operation','geonames_id','lat-lon', 'lat','lon','floor'
'expenses', 'properati_url', 'description', 'title',
'image_thumbnail','impRegex'`

Estas son eliminadas porque no aportan info al modelo y otras porque ya fueron analizadas y en base a ellas construimos otras columnas que contienen esa informacion relevante.

In [ ]:
df_properati2_drop1= df_properati2[['property_type','place_name','place_with_parent_names','country_name',
                                    'state_name','price','currency','price_aprox_local_currency','price_aprox_usd',
                                    'surface_total_in_m2','surface_covered_in_m2','price_usd_per_m2','price_per_m2',
                                    'rooms','priceUSD','priceUSDImp','amenities','piscina',
                                    'cochera','a_estrenar']]

En lo que refiere a datos de precio tambien podemos eliminar dado que en la etapa anterior se ha trabajado en la homogeneizacion de esta informacion, quedando la columna precio en dolares `'priceUSD'` y `'priceUSDImp'` como unificacion de los valores de los inmuebles. 

Columnas a eliminar:

`'price', 'currency','price_aprox_local_currency', 'price_aprox_usd'`

In [ ]:
df_properati2_drop2= df_properati2_drop1[['property_type','place_name','place_with_parent_names','country_name',
                                          'state_name','surface_total_in_m2','surface_covered_in_m2',
                                          'price_usd_per_m2','price_per_m2','rooms','priceUSD',
                                          'priceUSDImp','amenities','piscina','cochera','a_estrenar']]

Si recordamos, el problema ha quedado acotado a CABA por ello podemos reducir aun mas el DataFrame.

### Sacamos:

`'country_name'` solo tiene valores de Argentina no aporta info de CABA

`'place_with_parent_names'` la info que aporta esta contenida en la columna `'place_name'`

In [ ]:
df_properati2_drop3= df_properati2_drop2[['property_type','place_name','state_name',
                                          'surface_total_in_m2','surface_covered_in_m2','price_usd_per_m2',
                                          'price_per_m2','rooms','priceUSD','priceUSDImp',
                                          'amenities','piscina','cochera','a_estrenar']]

In [ ]:
df_properati_limpio_CABA= df_properati2_drop3[df_properati2_drop3.state_name=="Capital Federal"]

In [ ]:
df_properati_limpio_CABA

Revisamos los nulos

In [ ]:
df_properati_limpio_CABA.isnull().sum()

In [ ]:
df_properati_limpio_CABA.corr()

Vemos las correlaciones

In [ ]:
sns.heatmap(df_properati_limpio_CABA.corr(), linewidths=.5, annot=True)

Existen valores que llaman la atencion, en muchos casos podemos sospechar que vienen de los outliers y los valores de 1 es porque la info que contienen es practicamente la mismo como es el caso de `priceUSD` y  `priceUSDImp` 

### Nota:
Notamos que existen algunos registros que la variable `place_name` y `state_name` toman el mismo valor, por ello los eliminamos dado que no podemos saber a que barrio de CABA pertenecen.

In [ ]:
indice_drop_sbarrio= df_properati_limpio_CABA[df_properati_limpio_CABA.place_name=="Capital Federal"].index

In [ ]:
df_properati_limpio2_CABA= df_properati_limpio_CABA.drop(indice_drop_sbarrio, axis=0)

In [ ]:
df_properati_limpio2_CABA.isnull().sum()

### Eliminamos los nulos de la columna a predecir  `price_usd_per_m2`

In [ ]:
indice_drop_ppm2dol= df_properati_limpio2_CABA[df_properati_limpio2_CABA.price_usd_per_m2.isnull()].index

In [ ]:
df_properati_limpio3_CABA= df_properati_limpio2_CABA.drop(indice_drop_ppm2dol, axis=0)

In [ ]:
df_properati_limpio3_CABA.isnull().sum()

In [ ]:
df_properati_limpio3_CABA

In [ ]:
sns.heatmap(df_properati_limpio3_CABA[df_properati_limpio3_CABA.surface_total_in_m2<20000].corr(), linewidths=.5, annot=True)

#### Nota: Se ve que algunas correlaciones mejoran pero debemos seguir trabajando los datos.


### Comparamos `surface_covered_in_m2` y  `surface_total_in_m2`

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(15,7))
sns.scatterplot(x=df_properati_limpio3_CABA[df_properati_limpio3_CABA.surface_covered_in_m2<2000].rooms, y=df_properati_limpio3_CABA.surface_covered_in_m2, hue=df_properati_limpio3_CABA.property_type, ax=axs[0])

sns.scatterplot(x=df_properati_limpio3_CABA[df_properati_limpio3_CABA.surface_total_in_m2<25000].rooms, y=df_properati_limpio3_CABA.surface_total_in_m2, hue=df_properati_limpio3_CABA.property_type, ax=axs[1])

In [ ]:
df_properati_limpio3_CABA.columns

### Eliminamos 

`'state_name'` dado que el analisis es en CABA

`'surface_covered_in_m2'` esta info esta contenida en `'surface_total_in_m2'`

`'price_per_m2'` esta info esta contenida en `'price_usd_per_m2'`

`'rooms'` tiene demasiados nulos, consideramos que esta contenida en `'surface_total_in_m2'`, y ademas la corr con `price_usd_per_m2`es casi cero.

`'priceUSDImp'` esta info esta contenida en `'priceUSD'`


In [ ]:
df_properati_limpio4_CABA= df_properati_limpio3_CABA[['property_type', 'place_name', 
                                                      'surface_total_in_m2',
                                                     'price_usd_per_m2','priceUSD', 
                                                     'amenities','piscina','cochera','a_estrenar']]

In [ ]:
sns.heatmap(df_properati_limpio4_CABA.corr(), linewidths=.5, annot=True)

In [ ]:
df_properati_limpio4_CABA.isnull().sum()

In [ ]:
df_properati_limpio4_CABA.info()

In [ ]:
df_properati_limpio4_CABA.place_name.value_counts()

In [ ]:
sum(df_properati_limpio4_CABA.place_name.value_counts()>30)

### Eliminamos los valores duplicados

In [ ]:
df_properati_limpio4_CABA.duplicated().sum()

In [ ]:
df_properati_limpio4_CABA= df_properati_limpio4_CABA.drop_duplicates()

In [ ]:
df_properati_limpio4_CABA.shape

Hasta aqui llegamos con el dataset sin nulls con un shape de (20288, 9)  y con las variables que consideramos relevantes para comenzar a avaluar modelos.

`Data columns (total 9 columns):`

     `#Column               Non-Null Count  Dtype`  
 `0   property_type        22603 non-null  object 
  1   place_name           22603 non-null  object 
  2   surface_total_in_m2  22603 non-null  float64
  3   price_usd_per_m2     22603 non-null  float64
  4   priceUSD             22603 non-null  float64
  5   amenities            22603 non-null  object 
  6   piscina              22603 non-null  object 
  7   cochera              22603 non-null  object 
  8   a_estrenar           22603 non-null  object`


### Analisis de la columna `price_usd_per_m2` versus su calculo a partir de `priceUSD`/ `surface_total_in_m2`

Vamos a calcularlo en base a los datos de precio total en dolares y superficie total.

In [ ]:
precio_m2_dol_calc= df_properati_limpio4_CABA.apply(lambda x: x["priceUSD"]/x["surface_total_in_m2"], axis=1)

In [ ]:
precio_m2_dol_calc.sum()

In [ ]:
df_properati_limpio4_CABA.price_usd_per_m2.sum()

Notamos que hay una diferencia entre ambas opciones veamos si es significativa

In [ ]:
diff=abs(precio_m2_dol_calc - df_properati_limpio4_CABA.price_usd_per_m2)

In [ ]:
diff[diff>1].shape[0]/df_properati_limpio4_CABA.shape[0]*100

In [ ]:
diff[diff>1].index.shape[0]

La diferencia encontrada es menor al 5%. En valor de registros representan 887. En base a esta info decidimos dejar la columna que viene con el DataSet.

## Comenzamos a evaluar modelos


#### Escribimos una funcion que realiza todos los pasos: instancia, splitea, escala, entrena y calcula el R2

In [ ]:
def grupo_4_etapa2(df,var_x,var_y): # var_x es una lista de features y var_y el vector a predecir
    
    X = df[var_x]
    y=  df[var_y]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=53)
    scale          = StandardScaler()
    X_train_scaled = scale.fit_transform(X_train) 
    X_test_scaled  = scale.transform(X_test) 
    
    # les pongo el nombre a las columnas 
    X_train_scaled =  pd.DataFrame(X_train_scaled,columns=X.columns)
    X_test_scaled =  pd.DataFrame(X_test_scaled,columns=X.columns)
    
    lm = linear_model.LinearRegression()
    lm.fit(X_train_scaled,y_train)
    r2_test = lm.score(X_test_scaled, y_test)
    r2_train= lm.score(X_train_scaled, y_train)
    
    #print (f"Las variables X son", var_x, ", la variable y es", var_y)
    print ("El R2_test es:", r2_test )
    print ("El R2_train es:", r2_train)
        
    return lm, scale

#### Usando las variables numericas aproximamos buscando un valor global de R2 

In [ ]:
lm_limpio4, scale_limpio4 = grupo_4_etapa2(df_properati_limpio4_CABA, ['surface_total_in_m2','priceUSD'],['price_usd_per_m2'] )

#### Buscamos Outliers mirando graficos y el DataSet

In [ ]:
sns.distplot(df_properati_limpio4_CABA[df_properati_limpio4_CABA.surface_total_in_m2<500].surface_total_in_m2)

In [ ]:
df_properati_limpio4_CABA[df_properati_limpio4_CABA.surface_total_in_m2<27].shape

In [ ]:
df_properati_limpio4_CABA[df_properati_limpio4_CABA.surface_total_in_m2>500].shape

In [ ]:
sns.distplot(df_properati_limpio4_CABA[df_properati_limpio4_CABA.price_usd_per_m2<6000].price_usd_per_m2)

In [ ]:
df_properati_limpio4_CABA[df_properati_limpio4_CABA.price_usd_per_m2<900]

In [ ]:
sns.scatterplot(x=df_properati_limpio4_CABA.priceUSD[df_properati_limpio4_CABA.priceUSD<1000000], y=df_properati_limpio4_CABA.price_usd_per_m2)

### Nota: 
Se puede observar que existen valores que llamamos outliers para las columnas graficadas o sea las variables `'surface_total_in_m2', 'price_usd_per_m2', 'priceUSD'` tienen valores fuera de rango. Mas adelante en el analisis explicaremos los criterios adoptados para descartarlos en nuestro `DataFrame`

En relacion a los valores de `'priceUSD'` no vale la pena detenerse dado que mas adelante aclararemos porque la desestimamos.

#### Analizamos por barrio

In [ ]:
df_belgrano=df_properati_limpio4_CABA[df_properati_limpio4_CABA.place_name=="Belgrano"]
df_palermo=df_properati_limpio4_CABA[df_properati_limpio4_CABA.place_name=="Palermo"]
df_caballito=df_properati_limpio4_CABA[df_properati_limpio4_CABA.place_name=="Caballito"]

In [ ]:
lm_belgrano, scaler_belgrano = grupo_4_etapa2(df_belgrano, ['surface_total_in_m2','priceUSD'],['price_usd_per_m2'] )

In [ ]:
lm_palermo, scaler_palermo = grupo_4_etapa2(df_palermo, ['surface_total_in_m2','priceUSD'],['price_usd_per_m2'] )

In [ ]:
lm_caballito, scaler_caballito = grupo_4_etapa2(df_caballito, ['surface_total_in_m2','priceUSD'],['price_usd_per_m2'] )

#### Veamos Belgrano

In [ ]:
lm_belgrano_sinprice, scaler_belgrano_sinprice = grupo_4_etapa2(df_belgrano, ['surface_total_in_m2'],['priceUSD'] )

Buscamos Outliers

In [ ]:
sns.distplot(df_belgrano[df_belgrano.surface_total_in_m2<400].surface_total_in_m2)

Segun la grafica podemos acotar como representativo propiedades de hasta 400m2

In [ ]:
sns.scatterplot(x=df_belgrano.priceUSD, y=df_belgrano.price_usd_per_m2, hue=df_belgrano.property_type)

In [ ]:
sns.scatterplot(x=df_belgrano.surface_total_in_m2[df_belgrano.surface_total_in_m2<1500], y=df_belgrano.price_usd_per_m2, hue=df_belgrano.property_type)

In [ ]:
sns.scatterplot(x=df_caballito.surface_total_in_m2[df_caballito.surface_total_in_m2<1500], y=df_caballito.price_usd_per_m2, hue=df_caballito.property_type)

In [ ]:
df_properati_limpio4_CABA[df_properati_limpio4_CABA.surface_total_in_m2>300]

In [ ]:
df_palermo[df_palermo.surface_total_in_m2>300]

In [ ]:
df_belgrano_sub_ = df_belgrano[df_belgrano.surface_total_in_m2<200]

In [ ]:
lm_belgrano_sub, scaler_belgrano_sub = grupo_4_etapa2(df_belgrano_sub_, ['surface_total_in_m2'],['price_usd_per_m2'] )

In [ ]:
df_properati_limpio4_CABA.property_type.value_counts()

#### Analizamos ahora en funcion a la variable `property_type`

In [ ]:
df_apartment = df_properati_limpio4_CABA[df_properati_limpio4_CABA.property_type== "apartment"]

In [ ]:
df_PH = df_properati_limpio4_CABA[df_properati_limpio4_CABA.property_type== "PH"]

In [ ]:
df_house = df_properati_limpio4_CABA[df_properati_limpio4_CABA.property_type== "house"]

In [ ]:
lm_apartment, scaler_apartmen = grupo_4_etapa2(df_apartment, ['surface_total_in_m2'],['price_usd_per_m2'] )

In [ ]:
lm_house, scaler_house = grupo_4_etapa2(df_house, ['surface_total_in_m2'],['price_usd_per_m2'] )

In [ ]:
lm_PH, scaler_PH = grupo_4_etapa2(df_PH, ['surface_total_in_m2'],['price_usd_per_m2'] )

### Nota:
Luego de haber probado diferentes opciones de modelos segun barrio y segun tipo de propiedad, ahora transformamos las variables categoricas en numericas para poder incluirlas en el modelo de Regrsion Lineal Multiple.

### Transformamos `'amenities', 'piscina', 'cochera', 'a_estrenar'` en 1 y 0

In [ ]:
df_properati_limpio5_CABA = df_properati_limpio4_CABA.replace({"amenities":1, "piscina":1, "cochera":1, "a_estrenar":1, "False":0})
df_properati_limpio5_CABA

In [ ]:
df_properati_limpio5_CABA.info()

### Generamos las Dummys para ` "property_type" y "place_name"`

In [ ]:
property_type_dummies = pd.get_dummies(df_properati_limpio5_CABA.property_type, drop_first=True)
property_type_dummies

In [ ]:
place_name_dummies = pd.get_dummies(df_properati_limpio5_CABA.place_name, drop_first=True)

In [ ]:
place_name_dummies


#### Unimos ambos df

In [ ]:
df_properati_limpio5_CABA_con_dummys  = pd.concat([df_properati_limpio5_CABA, property_type_dummies, place_name_dummies], axis = 1)

In [ ]:
df_properati_limpio5_CABA_con_dummys

In [ ]:
df_properati_limpio5_CABA_con_dummys.info()

In [ ]:
df_properati_limpio5_CABA_con_dummys.columns

#### Calculamos el R2 de Train y Test del `DataFrame` con 20288 filas y 72 columnas

In [ ]:
lm_limpio5_condummys, scaler_limpio5condummys = grupo_4_etapa2(df_properati_limpio5_CABA_con_dummys,['surface_total_in_m2','amenities', 'piscina', 'cochera',
       'a_estrenar', 'apartment', 'house', 'store', 'Agronomía', 'Almagro',
       'Balvanera', 'Barracas', 'Barrio Norte', 'Belgrano', 'Boca', 'Boedo',
       'Caballito', 'Catalinas', 'Centro / Microcentro', 'Chacarita',
       'Coghlan', 'Colegiales', 'Congreso', 'Constitución', 'Flores',
       'Floresta', 'Las Cañitas', 'Liniers', 'Mataderos', 'Monserrat',
       'Monte Castro', 'Nuñez', 'Once', 'Palermo', 'Palermo Chico',
       'Palermo Hollywood', 'Palermo Soho', 'Palermo Viejo',
       'Parque Avellaneda', 'Parque Centenario', 'Parque Chacabuco',
       'Parque Chas', 'Parque Patricios', 'Paternal', 'Pompeya',
       'Puerto Madero', 'Recoleta', 'Retiro', 'Saavedra', 'San Cristobal',
       'San Nicolás', 'San Telmo', 'Tribunales', 'Velez Sarsfield',
       'Versalles', 'Villa Crespo', 'Villa Devoto', 'Villa General Mitre',
       'Villa Lugano', 'Villa Luro', 'Villa Ortuzar', 'Villa Pueyrredón',
       'Villa Real', 'Villa Riachuelo', 'Villa Santa Rita', 'Villa Soldati',
       'Villa Urquiza', 'Villa del Parque'], ['price_usd_per_m2'] )

**Notese que los valores de  R2 que obtuvimos son muy malos. La diferencia entre el de train y de test podria estar mostrando un posible sobreajuste.** 

### Como primer camino volveremos a la tarea de limpieza del DataSet para ver si mejoran nuestros R2

Veamos los boxplot de superficie y de precio por metro buscando outliers

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(30,10))
sns.boxplot(x=df_properati_limpio5_CABA_con_dummys.surface_total_in_m2, ax=axs[0])
sns.boxplot(x=df_properati_limpio5_CABA_con_dummys.price_usd_per_m2, ax=axs[1])

Horribles, trabajemos sobres eso.

### Feature `'surface_total_in_m2'`

En la etapa anterior vimos que teniamos outliers en la columna `'surface_total_in_m2'` tanto por exceso como por defecto.
Tomemos un criterio para determinar ambos limites.
En el caso del minimo segun lo visto por el reglamento de construccion de CABA para el periodo en analisis vimos que no esta permitido construir propiedades menores a **27m2**(ver1) de superficie. Y el limite superiro si bien puede ser un poco arbitrario nos parecio un criterio logico establecerlo en **10000m2.** dado que casi no hay observaciones.

ver1: https://www.infobae.com/politica/2018/09/06/la-legislatura-portena-avanzo-en-la-reforma-de-los-codigos-de-planeamiento-urbano-y-de-edificacion/



In [ ]:
indice_drop_sup_min= df_properati_limpio5_CABA_con_dummys[df_properati_limpio5_CABA_con_dummys.surface_total_in_m2<27].index

In [ ]:
df_properati_limpio6_CABA_con_dummys= df_properati_limpio5_CABA_con_dummys.drop(indice_drop_sup_min ,axis=0)

In [ ]:
df_properati_limpio6_CABA_con_dummys.shape

In [ ]:
indice_drop_sup_max= df_properati_limpio6_CABA_con_dummys[df_properati_limpio6_CABA_con_dummys.surface_total_in_m2>10000].index

In [ ]:
df_properati_limpio7_CABA_con_dummys= df_properati_limpio6_CABA_con_dummys.drop(indice_drop_sup_max ,axis=0)

In [ ]:
df_properati_limpio7_CABA_con_dummys.shape

In [ ]:
lm_limpio7_condummys, scaler_limpio7_condummys = grupo_4_etapa2(df_properati_limpio7_CABA_con_dummys,['surface_total_in_m2','amenities', 'piscina', 'cochera',
       'a_estrenar', 'apartment', 'house', 'store', 'Agronomía', 'Almagro',
       'Balvanera', 'Barracas', 'Barrio Norte', 'Belgrano', 'Boca', 'Boedo',
       'Caballito', 'Catalinas', 'Centro / Microcentro', 'Chacarita',
       'Coghlan', 'Colegiales', 'Congreso', 'Constitución', 'Flores',
       'Floresta', 'Las Cañitas', 'Liniers', 'Mataderos', 'Monserrat',
       'Monte Castro', 'Nuñez', 'Once', 'Palermo', 'Palermo Chico',
       'Palermo Hollywood', 'Palermo Soho', 'Palermo Viejo',
       'Parque Avellaneda', 'Parque Centenario', 'Parque Chacabuco',
       'Parque Chas', 'Parque Patricios', 'Paternal', 'Pompeya',
       'Puerto Madero', 'Recoleta', 'Retiro', 'Saavedra', 'San Cristobal',
       'San Nicolás', 'San Telmo', 'Tribunales', 'Velez Sarsfield',
       'Versalles', 'Villa Crespo', 'Villa Devoto', 'Villa General Mitre',
       'Villa Lugano', 'Villa Luro', 'Villa Ortuzar', 'Villa Pueyrredón',
       'Villa Real', 'Villa Riachuelo', 'Villa Santa Rita', 'Villa Soldati',
       'Villa Urquiza', 'Villa del Parque'], ['price_usd_per_m2'] )

Vemos que limpiando los outliers de la columna `'surface_total_in_m2'` se produce una mejora en la relacion entre train y test pero aun sigue siendo bajo el R2 de ambos. Trabajemos ahora con el Feature `'price_usd_per_m2'`

### Feature `'price_usd_per_m2'`

Anteriormente vimos que teniamos outliers en la columna `'price_usd_per_m2'` tanto por exceso como por defecto.
Tomemos un criterio para determinar ambos limites.
En el caso del minimo segun lo visto en publicaiones sobre el mercado de propiedades de CABA podemos establecer que para el periodo en analisis no encontramos valores de esta variable inferiores a **900 uSs/m2.** En referencia al limite superior y efectuando el mismo analisis podemos establecer que buscando la zona mas cara vemos que no se supera el valor de **6000 uSs/m2** 

ver: http://www.maureinmobiliaria.com/metro-cuadrado/


In [ ]:
indice_drop_price_m2_min= df_properati_limpio7_CABA_con_dummys[df_properati_limpio7_CABA_con_dummys.price_usd_per_m2>6000].index

In [ ]:
df_properati_limpio8_CABA_con_dummys= df_properati_limpio7_CABA_con_dummys.drop(indice_drop_price_m2_min ,axis=0)

In [ ]:
df_properati_limpio8_CABA_con_dummys.shape

In [ ]:
indice_drop_price_m2_max= df_properati_limpio8_CABA_con_dummys[df_properati_limpio8_CABA_con_dummys.price_usd_per_m2<900].index

In [ ]:
df_properati_limpio9_CABA_con_dummys= df_properati_limpio8_CABA_con_dummys.drop(indice_drop_price_m2_max ,axis=0)

In [ ]:
df_properati_limpio9_CABA_con_dummys.shape

#### Resumen criterio para descartar Outliers

`surface_total_in_m2`

                419 row por sup_m2<27

                2 row por sup_m2>10000

`price_usd_per_m2`

                955 row por precio_m2>6000

                383 row por precio_m2<900

#### *Total de observaciones descartadas 1759row*



Vemos como quedaron los boxplot ahora

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(30,10))
sns.boxplot(x=df_properati_limpio9_CABA_con_dummys[df_properati_limpio9_CABA_con_dummys.surface_total_in_m2<10000].surface_total_in_m2, ax=axs[0])
sns.boxplot(x=df_properati_limpio9_CABA_con_dummys.price_usd_per_m2, ax=axs[1])

**Notese** que en superficie podes eliminar algunos valores mas pero no afectan a nuestro R2. Por el momento los dejamos.

### Recalculamos los R2

In [ ]:
lm_limpio9_condummys, scaler_limpio9_condummys = grupo_4_etapa2(df_properati_limpio9_CABA_con_dummys,['surface_total_in_m2', 'amenities', 'piscina', 'cochera',
       'a_estrenar', 'apartment', 'house', 'store', 'Agronomía', 'Almagro',
       'Balvanera', 'Barracas', 'Barrio Norte', 'Belgrano', 'Boca', 'Boedo',
       'Caballito', 'Catalinas', 'Centro / Microcentro', 'Chacarita',
       'Coghlan', 'Colegiales', 'Congreso', 'Constitución', 'Flores',
       'Floresta', 'Las Cañitas', 'Liniers', 'Mataderos', 'Monserrat',
       'Monte Castro', 'Nuñez', 'Once', 'Palermo', 'Palermo Chico',
       'Palermo Hollywood', 'Palermo Soho', 'Palermo Viejo',
       'Parque Avellaneda', 'Parque Centenario', 'Parque Chacabuco',
       'Parque Chas', 'Parque Patricios', 'Paternal', 'Pompeya',
       'Puerto Madero', 'Recoleta', 'Retiro', 'Saavedra', 'San Cristobal',
       'San Nicolás', 'San Telmo', 'Tribunales', 'Velez Sarsfield',
       'Versalles', 'Villa Crespo', 'Villa Devoto', 'Villa General Mitre',
       'Villa Lugano', 'Villa Luro', 'Villa Ortuzar', 'Villa Pueyrredón',
       'Villa Real', 'Villa Riachuelo', 'Villa Santa Rita', 'Villa Soldati',
       'Villa Urquiza', 'Villa del Parque'], ['price_usd_per_m2'] )

In [ ]:
df = df_properati_limpio9_CABA_con_dummys.copy()
var_x = ['surface_total_in_m2', 'amenities', 'piscina', 'cochera',
       'a_estrenar', 'apartment', 'house', 'store', 'Agronomía', 'Almagro',
       'Balvanera', 'Barracas', 'Barrio Norte', 'Belgrano', 'Boca', 'Boedo',
       'Caballito', 'Catalinas', 'Centro / Microcentro', 'Chacarita',
       'Coghlan', 'Colegiales', 'Congreso', 'Constitución', 'Flores',
       'Floresta', 'Las Cañitas', 'Liniers', 'Mataderos', 'Monserrat',
       'Monte Castro', 'Nuñez', 'Once', 'Palermo', 'Palermo Chico',
       'Palermo Hollywood', 'Palermo Soho', 'Palermo Viejo',
       'Parque Avellaneda', 'Parque Centenario', 'Parque Chacabuco',
       'Parque Chas', 'Parque Patricios', 'Paternal', 'Pompeya',
       'Puerto Madero', 'Recoleta', 'Retiro', 'Saavedra', 'San Cristobal',
       'San Nicolás', 'San Telmo', 'Tribunales', 'Velez Sarsfield',
       'Versalles', 'Villa Crespo', 'Villa Devoto', 'Villa General Mitre',
       'Villa Lugano', 'Villa Luro', 'Villa Ortuzar', 'Villa Pueyrredón',
       'Villa Real', 'Villa Riachuelo', 'Villa Santa Rita', 'Villa Soldati',
       'Villa Urquiza', 'Villa del Parque']
var_y = ['price_usd_per_m2']

X = df[var_x]
y=  df[var_y]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=53)

X_train_scaled = scaler_limpio9_condummys.transform(X_train)
X_test_scaled = scaler_limpio9_condummys.transform(X_test)

kf = KFold(n_splits=5, shuffle=True, random_state=12)

In [ ]:
lm_lasso =  linear_model.LassoCV(alphas=np.linspace(0.01,100, 1000), cv=kf, normalize=True)

reg_lineal_lasso = lm_lasso.fit(X_train_scaled, np.array(y_train).ravel())

score_lasso_precio = reg_lineal_lasso.score(X_test_scaled, np.array(y_test).ravel() )

print('Score del modelo Lasso:', score_lasso_precio)

## Podemos probar aplicando transformaciones a la variable `price_usd_per_m2` para ver si mejora el R2.

### Probemos trasformando la variable "y"=`price_usd_per_m2` con logaritmos 

In [ ]:
df_properati_limpio9_CABA_con_dummys.columns

In [ ]:
price_usd_per_m2_log =np.log(df_properati_limpio9_CABA_con_dummys.price_usd_per_m2)
df_properati_limpio9_CABA_con_dummys_log= df_properati_limpio9_CABA_con_dummys.copy()
df_properati_limpio9_CABA_con_dummys_log["price_usd_per_m2_log"]= price_usd_per_m2_log

In [ ]:
a4_dims = (11.7,8.3)
fig, ax = plt.subplots(figsize=a4_dims)
sns.distplot(df_properati_limpio9_CABA_con_dummys_log["price_usd_per_m2_log"],color='red')

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.distplot(df_properati_limpio9_CABA_con_dummys_log["price_usd_per_m2"], color= 'red')

In [ ]:
df_properati_limpio9_CABA_con_dummys_log.columns

In [ ]:
lm_limpio9_condummys_log, scaler_limpio9_condummys_log = grupo_4_etapa2(df_properati_limpio9_CABA_con_dummys_log,['surface_total_in_m2', 'amenities', 'piscina', 'cochera',
       'a_estrenar', 'apartment', 'house', 'store', 'Agronomía', 'Almagro',
       'Balvanera', 'Barracas', 'Barrio Norte', 'Belgrano', 'Boca', 'Boedo',
       'Caballito', 'Catalinas', 'Centro / Microcentro', 'Chacarita',
       'Coghlan', 'Colegiales', 'Congreso', 'Constitución', 'Flores',
       'Floresta', 'Las Cañitas', 'Liniers', 'Mataderos', 'Monserrat',
       'Monte Castro', 'Nuñez', 'Once', 'Palermo', 'Palermo Chico',
       'Palermo Hollywood', 'Palermo Soho', 'Palermo Viejo',
       'Parque Avellaneda', 'Parque Centenario', 'Parque Chacabuco',
       'Parque Chas', 'Parque Patricios', 'Paternal', 'Pompeya',
       'Puerto Madero', 'Recoleta', 'Retiro', 'Saavedra', 'San Cristobal',
       'San Nicolás', 'San Telmo', 'Tribunales', 'Velez Sarsfield',
       'Versalles', 'Villa Crespo', 'Villa Devoto', 'Villa General Mitre',
       'Villa Lugano', 'Villa Luro', 'Villa Ortuzar', 'Villa Pueyrredón',
       'Villa Real', 'Villa Riachuelo', 'Villa Santa Rita', 'Villa Soldati',
       'Villa Urquiza', 'Villa del Parque'], ['price_usd_per_m2_log'] )

Tambien probamos aplicando sqrt y vemos que el R2 es casi el mismo.

### Probemos con Box-Cox

In [ ]:
from scipy import stats

In [ ]:
x=stats.boxcox(df_properati_limpio9_CABA_con_dummys.price_usd_per_m2)
x

Tomo el lambda que maximiza la función log-verosimilitud. `lmbda=0.09122257603211358`

In [ ]:
x2=stats.boxcox(df_properati_limpio9_CABA_con_dummys.price_usd_per_m2, lmbda=0.09122257603211358)
x2.shape

In [ ]:
price_usd_per_m2_BCox= stats.boxcox(df_properati_limpio9_CABA_con_dummys.price_usd_per_m2, lmbda=0.09122257603211358)
df_properati_limpio9_CABA_con_dummys_BCox= df_properati_limpio9_CABA_con_dummys.copy()
df_properati_limpio9_CABA_con_dummys_BCox["price_usd_per_m2_BCox"]= price_usd_per_m2_BCox

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.distplot(df_properati_limpio9_CABA_con_dummys_BCox["price_usd_per_m2_BCox"],color='red')

In [ ]:
lm_limpio9_condummys_BCox, scaler_limpio9_condummys_BCox = grupo_4_etapa2(df_properati_limpio9_CABA_con_dummys_BCox,['surface_total_in_m2', 'amenities', 'piscina', 'cochera',
       'a_estrenar', 'apartment', 'house', 'store', 'Agronomía', 'Almagro',
       'Balvanera', 'Barracas', 'Barrio Norte', 'Belgrano', 'Boca', 'Boedo',
       'Caballito', 'Catalinas', 'Centro / Microcentro', 'Chacarita',
       'Coghlan', 'Colegiales', 'Congreso', 'Constitución', 'Flores',
       'Floresta', 'Las Cañitas', 'Liniers', 'Mataderos', 'Monserrat',
       'Monte Castro', 'Nuñez', 'Once', 'Palermo', 'Palermo Chico',
       'Palermo Hollywood', 'Palermo Soho', 'Palermo Viejo',
       'Parque Avellaneda', 'Parque Centenario', 'Parque Chacabuco',
       'Parque Chas', 'Parque Patricios', 'Paternal', 'Pompeya',
       'Puerto Madero', 'Recoleta', 'Retiro', 'Saavedra', 'San Cristobal',
       'San Nicolás', 'San Telmo', 'Tribunales', 'Velez Sarsfield',
       'Versalles', 'Villa Crespo', 'Villa Devoto', 'Villa General Mitre',
       'Villa Lugano', 'Villa Luro', 'Villa Ortuzar', 'Villa Pueyrredón',
       'Villa Real', 'Villa Riachuelo', 'Villa Santa Rita', 'Villa Soldati',
       'Villa Urquiza', 'Villa del Parque'], ['price_usd_per_m2_BCox'] )

Estos valores de R2 podrian considerarse bastante mejor teniendo en cuenta desde donde partimos. Ahora toca el turno de ver con regularizacion

## Ridge

In [ ]:
df = df_properati_limpio9_CABA_con_dummys.copy()
var_x = ['surface_total_in_m2', 'amenities', 'piscina', 'cochera',
       'a_estrenar', 'apartment', 'house', 'store', 'Agronomía', 'Almagro',
       'Balvanera', 'Barracas', 'Barrio Norte', 'Belgrano', 'Boca', 'Boedo',
       'Caballito', 'Catalinas', 'Centro / Microcentro', 'Chacarita',
       'Coghlan', 'Colegiales', 'Congreso', 'Constitución', 'Flores',
       'Floresta', 'Las Cañitas', 'Liniers', 'Mataderos', 'Monserrat',
       'Monte Castro', 'Nuñez', 'Once', 'Palermo', 'Palermo Chico',
       'Palermo Hollywood', 'Palermo Soho', 'Palermo Viejo',
       'Parque Avellaneda', 'Parque Centenario', 'Parque Chacabuco',
       'Parque Chas', 'Parque Patricios', 'Paternal', 'Pompeya',
       'Puerto Madero', 'Recoleta', 'Retiro', 'Saavedra', 'San Cristobal',
       'San Nicolás', 'San Telmo', 'Tribunales', 'Velez Sarsfield',
       'Versalles', 'Villa Crespo', 'Villa Devoto', 'Villa General Mitre',
       'Villa Lugano', 'Villa Luro', 'Villa Ortuzar', 'Villa Pueyrredón',
       'Villa Real', 'Villa Riachuelo', 'Villa Santa Rita', 'Villa Soldati',
       'Villa Urquiza', 'Villa del Parque']
var_y = ['price_usd_per_m2']

X = df[var_x]
y=  df[var_y]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=53)

X_train_scaled = scaler_limpio9_condummys.transform(X_train)
X_test_scaled = scaler_limpio9_condummys.transform(X_test)

kf = KFold(n_splits=5, shuffle=True, random_state=12)

lm_ridge =  linear_model.RidgeCV(alphas= np.linspace(0.1, 1, 100), cv=kf, normalize=True, scoring='r2')

reg_lineal_ridge = lm_ridge.fit(X_train_scaled, y_train)
reg_lineal_ridge

In [ ]:
reg_lineal_ridge.alpha_

In [ ]:
score_ridge_precio = reg_lineal_ridge.score(X_test_scaled, y_test)

lm_limpio9_condummys, scaler_limpio9_condummys

print('Score del modelo Ridge:', score_ridge_precio)
print(f'Score del modelo lineal {lm_limpio9_condummys.score(X_test_scaled,y_test)}')

In [ ]:
reg_lineal_ridge.intercept_

In [ ]:
lm_limpio9_condummys.intercept_

In [ ]:
X_train_scaled.shape

## Lasso

In [ ]:
guille = np.array(y_train).reshape(-1,1)
guille

guille2 = np.array(y_test).reshape(-1,1)
guille2.ravel()


In [ ]:
lm_lasso =  linear_model.LassoCV(alphas=np.linspace(0.01,100, 1000), cv=kf, normalize=True)

reg_lineal_lasso = lm_lasso.fit(X_train_scaled, guille.ravel())

score_lasso_precio = reg_lineal_lasso.score(X_test_scaled, guille2.ravel())

print('Score del modelo Lasso:', score_lasso_precio)

# Vemos los parámetros con StatModels


In [ ]:
var_x = ['surface_total_in_m2', 'amenities', 'piscina', 'cochera',
       'a_estrenar', 'apartment', 'house', 'store', 'Agronomía', 'Almagro',
       'Balvanera', 'Barracas', 'Barrio Norte', 'Belgrano', 'Boca', 'Boedo',
       'Caballito', 'Catalinas', 'Centro / Microcentro', 'Chacarita',
       'Coghlan', 'Colegiales', 'Congreso', 'Constitución', 'Flores',
       'Floresta', 'Las Cañitas', 'Liniers', 'Mataderos', 'Monserrat',
       'Monte Castro', 'Nuñez', 'Once', 'Palermo', 'Palermo Chico',
       'Palermo Hollywood', 'Palermo Soho', 'Palermo Viejo',
       'Parque Avellaneda', 'Parque Centenario', 'Parque Chacabuco',
       'Parque Chas', 'Parque Patricios', 'Paternal', 'Pompeya',
       'Puerto Madero', 'Recoleta', 'Retiro', 'Saavedra', 'San Cristobal',
       'San Nicolás', 'San Telmo', 'Tribunales', 'Velez Sarsfield',
       'Versalles', 'Villa Crespo', 'Villa Devoto', 'Villa General Mitre',
       'Villa Lugano', 'Villa Luro', 'Villa Ortuzar', 'Villa Pueyrredón',
       'Villa Real', 'Villa Riachuelo', 'Villa Santa Rita', 'Villa Soldati',
       'Villa Urquiza', 'Villa del Parque']
var_y = ['price_usd_per_m2']
df = df_properati_limpio9_CABA_con_dummys.copy()


X = df[var_x]
y=  df[var_y]

print(X.shape)
print(y.shape)
       
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,  random_state=53)
scale = StandardScaler()
X_train_scaled_ori = scale.fit_transform(X_train) 
X_test_scaled_ori  = scale.transform(X_test) 

print(X_train_scaled_ori.shape)
print(X_test_scaled_ori.shape)


X_train_scaled_ori_1 = sm.add_constant(X_train_scaled_ori)
# X_test_scaled_ori_1 = sm.add_constant(X_test_scaled_ori)
X_test_scaled_ori_1 = sm.add_constant(X_test_scaled_ori, has_constant="add")


print(X_train_scaled_ori_1.shape)
print(X_test_scaled_ori_1.shape)


# les pongo el nombre a las columnas 
columnas = X.columns
print(columnas.shape)
columnas = np.insert(columnas, 0 , 'constant')
X_train_scaled =  pd.DataFrame(X_train_scaled_ori_1,columns=columnas)
X_test_scaled =  pd.DataFrame(X_test_scaled_ori_1,columns=columnas)
    
    
# Igualo el índice de y_train al de X_train_scaled
y_train = y_train.reset_index().drop('index', axis=1)
    
model = sm.OLS(y_train, X_train_scaled).fit()
predictions = model.predict(X_test_scaled)
print(model.summary())

## Aca comenzamos a tomar la muestra de 100 propiedades

Tomamos un Sample de 100 propiedades que conformarian el portafolio de los inmuebles sobre los cuales vamos a tomar la decisión de vender o no vender.

In [ ]:
testeo = df_properati_limpio9_CABA_con_dummys.copy()

#armo el sample
prueba_pred = testeo.sample(100, random_state=2).copy()

#armo los vectores para el modelo
X = prueba_pred.drop(['property_type', 'place_name','price_usd_per_m2','priceUSD'], axis = 1)
y = prueba_pred['price_usd_per_m2']

#predigo con el modelo lineal el precio x m2
prediccion = lm_limpio9_condummys.predict(scaler_limpio9_condummys.transform(X))
# prediccion_Lasso = lm_lasso.predict(scale.transform(X))
# prediccion_Ridge = lm_ridge.predict(scale.transform(X))

#inserto la columna con el precio predicho
prueba_pred.insert(4,"precio_pred_RL", prediccion, True)
# prueba_pred.insert(4,"precio_pred_Lasso", prediccion_Lasso, True)
# prueba_pred.insert(4,"precio_pred_Ridge", prediccion_Ridge, True)

prueba_pred.head(50)


Inserto la columna con la diferencia entre el precio publicado con el predicho en el paso anterior

In [ ]:
dif =prueba_pred['price_usd_per_m2'] - prueba_pred['precio_pred_RL']
prueba_pred.insert(5,"Dif", dif, True)
prueba_pred

Acá analizamos si el precio al que se publicó es mayor al predicho siginifica que la propiedad está sobre valuada y por ende conviene vender.

Para filtrarlas armamos una mascara donde el campo `'Dif'` sea mayor a cero porque `dif =prueba_pred['price_usd_per_m2'] - prueba_pred['precio_pred_RL']`

In [ ]:
Prop_a_Vender = prueba_pred['Dif'] > 0

Vender = prueba_pred[Prop_a_Vender]

Ganancia = (Vender['Dif']*Vender['surface_total_in_m2']).sum()
print(f'La ganancia teórica por vender las propiedades sería: {Ganancia.round(2)}')

# creo el capital inicial el cual es la suma del portafolio incial de 100 propiedades
Capital_Inicial = prueba_pred['priceUSD'].sum()

# creo el vector de features y objetivo en base a todo nuestro dataset
X_Ds = testeo.drop(['property_type', 'place_name','price_usd_per_m2','priceUSD'], axis = 1)
y_Ds = testeo['price_usd_per_m2']

# hago la predicción de todo
prediccion_todo = lm_limpio9_condummys.predict(scaler_limpio9_condummys.transform(X_Ds))

# inserto la nueva columna con el precio predicho
testeo.insert(4,"precio_pred_RL", prediccion_todo, True)

# inserto la columna con la diferencia entre real y predicho
dif =testeo['price_usd_per_m2'] - testeo['precio_pred_RL']
testeo.insert(5,"Diferencia_RL", dif, True)

In [ ]:

testeo2 = testeo.copy()

testeo2.sort_values(by = ['Diferencia_RL'], ascending=True, inplace=True)

testeo2.reset_index(inplace=True)

testeo2

Creo la cartera de las propiedades que más diferencia tienen entre real y predicho para así elegir las que más nos conviene comprar para revender sin exceder el capital inicial que obtuvimos de nuestro sample de 100 propiedades.

In [ ]:
# creo una cartera vacia
Cartera = pd.DataFrame()

total = 0
x=0

# por cada propiedad en el df testeo2 voy incrementado mi gasto de capital inicial
for i in testeo2:
    if total < Capital_Inicial:
        Cartera = Cartera.append(testeo2.iloc[x])
        total += testeo2.iloc[x].priceUSD
        x+=1
total

In [ ]:
# estas son mi propiedades
Cartera

In [ ]:
Cartera.place_name.value_counts()

# A partir de aquí son solo coneptos

## Estuvimos probando otras formas de armar el regresor incluyendo o quitando otros features

Veamos que ocurre si sacamos algunas zonas con muy pocas observaciones

In [ ]:
df_properati_limpio9_CABA_con_dummys_log.place_name.value_counts()[df_properati_limpio9_CABA_con_dummys_log.place_name.value_counts()<50]#.index

Villa Soldati(1) y Pompeya (19) se podrian unificar

Villa Riachuelo (2) podria unificarse con Villa Lugano (90)

Catalinas (1) podria unificarse con Retiro (154)


### Pruebo sin Soldati, VRiachuelo y Catalinas

In [ ]:
lm_sinbarrios, scaler_sinbarrios = grupo_4_etapa2(df_properati_limpio9_CABA_con_dummys_log,['surface_total_in_m2', 'amenities', 'piscina', 'cochera',
       'a_estrenar', 'apartment', 'house', 'store', 'Agronomía', 'Almagro',
       'Balvanera', 'Barracas', 'Barrio Norte', 'Belgrano', 'Boca', 'Boedo',
       'Caballito', 'Centro / Microcentro', 'Chacarita',
       'Coghlan', 'Colegiales', 'Congreso', 'Constitución', 'Flores',
       'Floresta', 'Las Cañitas', 'Liniers', 'Mataderos', 'Monserrat',
       'Monte Castro', 'Nuñez', 'Once', 'Palermo', 'Palermo Chico',
       'Palermo Hollywood', 'Palermo Soho', 'Palermo Viejo',
       'Parque Avellaneda', 'Parque Centenario', 'Parque Chacabuco',
       'Parque Chas', 'Parque Patricios', 'Paternal', 'Pompeya',
       'Puerto Madero', 'Recoleta', 'Retiro', 'Saavedra', 'San Cristobal',
       'San Nicolás', 'San Telmo', 'Tribunales', 'Velez Sarsfield',
       'Versalles', 'Villa Crespo', 'Villa Devoto', 'Villa General Mitre',
       'Villa Lugano', 'Villa Luro', 'Villa Ortuzar', 'Villa Pueyrredón',
       'Villa Real', 'Villa Santa Rita',
       'Villa Urquiza', 'Villa del Parque'], ['price_usd_per_m2_log'] )

### Vuelvo a analizar el DF con rooms

In [ ]:
df_properati_limpio9_CABA_con_dummys.columns

In [ ]:
df_properati_limpio3_CABA.columns

In [ ]:
df_properati_limpio10_CABA_con_dummys=pd.concat([df_properati_limpio9_CABA_con_dummys, df_properati_limpio3_CABA.rooms], axis=1)
df_new= df_properati_limpio10_CABA_con_dummys.copy()

In [ ]:
df_new.info()

In [ ]:
indice_drop_null_rooms=df_new[df_new.rooms.isnull()].index

In [ ]:
df_new_con_rooms=df_new.drop(indice_drop_null_rooms, axis=0)
df_new_con_rooms.info()

Tomamos el index de cualquier barrio para borrar los null de rooms, por ejemplo Saavedra

In [ ]:
indice_drop_null_Saavedra=df_new_con_rooms[df_new_con_rooms.Saavedra.isnull()].index
indice_drop_null_Saavedra

In [ ]:
df_new_con_rooms2=df_new_con_rooms.drop(indice_drop_null_Saavedra, axis=0)
df_new_con_rooms2.info()

In [ ]:
df_new_con_rooms2.shape

In [ ]:
sns.scatterplot(x=df_new_con_rooms2.surface_total_in_m2, y=df_new_con_rooms2.price_usd_per_m2, hue=df_new_con_rooms2.property_type)

In [ ]:
df_new_con_rooms2.columns

In [ ]:
lm_conrooms, scaler_conrooms = grupo_4_etapa2(df_new_con_rooms2, ['surface_total_in_m2','amenities', 'piscina', 'cochera','a_estrenar', 'apartment', 
                                    'house', 'store', 
        'Agronomía', 'Almagro','Balvanera', 'Barracas', 'Barrio Norte', 'Belgrano', 'Boca', 'Boedo','Caballito', 
        'Catalinas','Centro / Microcentro', 'Chacarita','Coghlan', 'Colegiales', 'Congreso', 'Constitución', 'Flores',
        'Floresta', 'Las Cañitas', 'Liniers', 'Mataderos', 'Monserrat','Monte Castro', 'Nuñez', 'Once', 'Palermo', 
        'Palermo Chico','Palermo Hollywood', 'Palermo Soho', 'Palermo Viejo','Parque Avellaneda', 'Parque Centenario', 
        'Parque Chacabuco','Parque Chas', 'Parque Patricios', 'Paternal', 'Pompeya','Puerto Madero', 'Recoleta', 
        'Retiro', 'Saavedra', 'San Cristobal','San Nicolás', 'San Telmo', 'Tribunales', 'Velez Sarsfield',
        'Versalles', 'Villa Crespo', 'Villa Devoto', 'Villa General Mitre','Villa Lugano', 'Villa Luro', 
        'Villa Ortuzar', 'Villa Pueyrredón','Villa Real', 'Villa Riachuelo', 'Villa Santa Rita', 'Villa Soldati',
        'Villa Urquiza', 'Villa del Parque', 'rooms'], ['price_usd_per_m2'])